In [11]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",#"defog/sqlcoder-7b-2",#"bartowski/gemma-2-9b-it-GGUF",
	filename="Meta-Llama-3.1-8B-Instruct-Q6_K.gguf",#"sqlcoder-7b-q5_k_m.gguf",#"gemma-2-9b-it-Q5_K_S.gguf",
 n_ctx=4048,
 verbose=False,
 n_gpu_layers=-1
)

In [ ]:
llm_response = llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is your purpose, in 150 words ?"
		}
	],
    stream=True  # Active le streaming
)

# Afficher les tokens au fur et à mesure de leur génération
nb_tokens = 0
for token in llm_response:
    nb_tokens+=1
    content = token['choices'][0].get('delta', {}).get('content', '')
    print(content, end='', flush=True)


My purpose is to assist and provide information to users through text-based conversations. I can process and respond to a wide range of questions and topics, from general knowledge and entertainment to specific domains such as science, history, and culture. 

I aim to provide accurate and reliable information, as well as offer helpful suggestions and ideas. My responses are generated based on my training data, which includes a massive corpus of text from various sources. 

I can also engage in creative conversations, generate text, and even provide language translation. My ultimate goal is to assist users in finding the information they need, answering their questions, and providing helpful insights. Whether it's for personal or professional use, I strive to be a helpful and reliable tool for users.150


In [5]:
print(nb_tokens)

150


In [6]:
chat_history = [{"role": "system", "content": "### Instruction:\n1/5 - Ta t\u00e2che est de r\u00e9pondre aux questions \u00e0 venir sous forme de phrases g\u00e9n\u00e9r\u00e9es par des requ\u00eates SQL DuckDB valides sur la base des inputs ci-dessous. \n2/5 - Le SQL \u00eatre con\u00e7u pour g\u00e9n\u00e9rer une chaine de caract\u00e8res (un seul r\u00e9sultat dans une colonne nomm\u00e9e \"ANSWER\"), pour cela il faudra en g\u00e9n\u00e9ral agr\u00e9ger des select via || pour int\u00e9grer les valeurs au sein de phrases affichables en string.\n3/5 - Tu mettras en commentaire (via \"/* */\" et pas \"--\") avant le sql : entre les balises #COMP_DEBUT# et #COMP_FIN# ce que tu as compris du fichier \u00ab Les donn\u00e9es portent sur ... \u00bb puis entre #ATTENDU_DEBUT# et #ATTENDU_FIN# ce qu'on attend de la requ\u00eate en pr\u00e9fixant par \"Objet de la requ\u00eate propos\u00e9e : \".\n4/5 - Dans le commentaire ajoute entre #THEME_DEBUT# et #THEME_FIN# la th\u00e9matique de la requ\u00eate entre crochets avec 3 mots max, exemples : [Egalit\u00e9 des sexes], [Changement climatique], [Politique publique XXXX] etc.\n5/5 - ENFIN, apr\u00e8s le commentaire (apr\u00e8s */), encadre le sql par #SQL_DEBUT# et #SQL_FIN#.\n### Inputs:\n#### DDL de la table :\nCREATE TABLE example_dataset (\"IdAttribuant\" VARCHAR NULL, \"NomAttribuant\" VARCHAR NULL, \"Annee\" VARCHAR NULL, \"CodArticle\" VARCHAR NULL, \"CodInvFonc\" VARCHAR NULL, \"CodNatJurBenefCA\" VARCHAR NULL, \"LibOrgaBenef\" VARCHAR NULL, \"MtSubv\" VARCHAR NULL, \"LibPrestaNat\" VARCHAR NULL);\n#### Aper\u00e7u du contenu :\n     IdAttribuant       NomAttribuant  ...     MtSubv LibPrestaNat\n0  23750007900312  R\u00e9gion \u00cele-de-F...  ...    32000.0             \n1  23750007900312  R\u00e9gion \u00cele-de-F...  ...    10689.0             \n2  23750007900312  R\u00e9gion \u00cele-de-F...  ...     492.51             \n3  23750007900312  R\u00e9gion \u00cele-de-F...  ...    12000.0             \n4  23750007900312  R\u00e9gion \u00cele-de-F...  ...      158.6             \n5  23750007900312  R\u00e9gion \u00cele-de-F...  ...     1714.5             \n6  23750007900312  R\u00e9gion \u00cele-de-F...  ...  326645.43             \n7  23750007900312  R\u00e9gion \u00cele-de-F...  ...   657198.0             \n8  23750007900312  R\u00e9gion \u00cele-de-F...  ...    31072.5             \n9  23750007900312  R\u00e9gion \u00cele-de-F...  ...   340000.0             \n\n[10 rows x 9 columns]\n#### Synth\u00e8se via summarize :\n        column_name column_type  ...  count null_percentage\n0      IdAttribuant     VARCHAR  ...  22958            0.00\n1     NomAttribuant     VARCHAR  ...  22958            0.00\n2             Annee     VARCHAR  ...  22958            0.00\n3        CodArticle     VARCHAR  ...  22958           26.07\n4        CodInvFonc     VARCHAR  ...  22958           26.07\n5  CodNatJurBenefCA     VARCHAR  ...  22958            0.00\n6      LibOrgaBenef     VARCHAR  ...  22958            0.00\n7            MtSubv     VARCHAR  ...  22958            0.00\n8      LibPrestaNat     VARCHAR  ...  22958           99.99\n\n[9 rows x 12 columns]\n####Lien vers le dataset d\u2019origine :\nhttps://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/subventions-versees-chaque-annee-par-la-region-aux-associations/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B\n####Nombre de lignes du jeu de donn\u00e9es :\n22958\n####Persona :\nMets toi dans la peau d'un expert, approche datamining et pens\u00e9e off the box. \n####HINTS SQL :\n- Juste du SQL pas de balise markdown ni autre balise de mise en forme. \n- Pas de caract\u00e8re '`'\n- Traite les dates via des cast en varchar puis des substring.\n- Quelques fois, des valeurs num\u00e9riques sont encore en varchar, caste les en double avant d'y appliquer des fonctions de calcul ou d'agr\u00e9gation.\n- Utiliser les double quote pour s\u00e9lectonner les colonnes afin d\u2019\u00e9viter les probl\u00e8mes li\u00e9s aux caract\u00e8res sp\u00e9ciaux\n- Les alias de tes champs doivent \u00eatre simples, avec underscores, sans guillets\n- Les fonctions spatiales sont celles de l'extension SPATIAL de duckdb (st_*)\n- Pour de la carto, il vaut mieux passer les g\u00e9om\u00e9tries en wkt avec st_astext\n- Utilise le group by all si besoin d\u2019agr\u00e9ger\n- Utilise ST_Union_Agg  plut\u00f4t que ST COLLECT pour l\u2019agr\u00e9gation de g\u00e9om\u00e9tries\n- Les g\u00e9om\u00e9tries stock\u00e9es en struct se lisent avec st_geomfromgeojson\n- Dans les clauses where, les \u00e9ventuelles apostrophes sont a doubler plu\u00f4t que d\u2019utiliser un \u00e9chappement\u00a0: ex\u00a0: where modalite=\u2019vol d\\\u2019accessoire\u2019 doit \u00eatre where modalite=\u2019vol d\u2019\u2019accessoire\u2019 \n####HINTS REPONSES PHRASES (sp\u00e9cial chatbot):\n- Pour g\u00e9n\u00e9rer des phrases penses \u00e0 utiliser\u00a0: STRING_AGG pour concatener des occurrences\n- S\u00e9curise avec des distincts sur tes sous requ\u00eates avant d\u2019agr\u00e9g\u00e9r\n- Utilise des concat\u00e9nations de colonnes avec ||(select ...)||\n- Utilise group by all si tu utilise une fonction d\u2019aggr\u00e9gation\n- Propose la visualisation des positions avec street view et la visualisation des g\u00e9om\u00e9tries avec geojson.io\n###REPONSE DE DEPART (adapte la partie XXXX et ajoute 3 suggestions compte tenu des entrants)\n\"Bonjour bienvenue pour l'analyse de vos donn\u00e9es relatives \u00e0 XXXX. Voici 3 suggestions de questions\u00a0: \" ET ATTEND D'ETRE SOLLICITE PAR UNE QUESTION AVANT DE GENERER DU SQL"}, {"role": "user", "content": "Hello"}, {"role": "assistant", "content": "Bonjour bienvenue pour l'analyse de vos donn\u00e9es relatives aux subventions vers\u00e9es chaque ann\u00e9e par la r\u00e9gion \u00cele-de-France aux associations. Voici 3 suggestions de questions : \n\n1. Quelle est la subvention totale vers\u00e9e par la r\u00e9gion \u00cele-de-France par ann\u00e9e ?\n2. Quelles sont les trois associations qui ont re\u00e7u les plus grandes subventions en 2023 ?\n3. Quelle est la subvention la plus \u00e9lev\u00e9e vers\u00e9e \u00e0 une seule association en 2022 et quel est le nom de cette association ?\n\nN'h\u00e9sitez pas \u00e0 poser la question de votre choix pour que je g\u00e9n\u00e8re un SQL pertinent pour vous !"}, {"role": "assistant", "content": "Bonjour bienvenue pour l'analyse de vos donn\u00e9es relatives aux subventions vers\u00e9es chaque ann\u00e9e par la r\u00e9gion \u00cele-de-France aux associations. Voici 3 suggestions de questions : \n\n1. Quelle est la subvention totale vers\u00e9e par la r\u00e9gion \u00cele-de-France par ann\u00e9e ?\n2. Quelles sont les trois associations qui ont re\u00e7u les plus grandes subventions en 2023 ?\n3. Quelle est la subvention la plus \u00e9lev\u00e9e vers\u00e9e \u00e0 une seule association en 2022 et quel est le nom de cette association ?\n\nN'h\u00e9sitez pas \u00e0 poser la question de votre choix pour que je g\u00e9n\u00e8re un SQL pertinent pour vous !"}, {"role": "user", "content": "Pour chacune de ces tranches de subventions (0-10000, 10000-100000, 100000-1000000, >1000000), trouve le plus gros bénéficiaire et indique le nombre de lignes de subventions touchées, et le montant total touché par ce bénéficaire. "}]


In [12]:
llm_response = llm.create_chat_completion(
	messages = chat_history,
    stream=True  # Active le streaming
)

nb_tokens = 0
for token in llm_response:
    nb_tokens+=1
    content = token['choices'][0].get('delta', {}).get('content', '')
    print(content, end='', flush=True)

**#COMP_DEBUT#**
La tâche consiste à analyser les données de subventions versées par la région Île-de-France aux associations, en particulier à identifier les bénéficiaires les plus importants pour chaque tranche de subventions.
La base de données contient les informations suivantes : IdAttribuant, NomAttribuant, Annee, CodArticle, CodInvFonc, CodNatJurBenefCA, LibOrgaBenef, MtSubv, LibPrestaNat
La tranche de subventions est définie comme suit : 0-10000, 10000-100000, 100000-1000000, >1000000
L'objectif est de trouver le bénéficiaire le plus important pour chaque tranche de subventions, ainsi que le nombre de lignes de subventions touchées et le montant total touché.
**#THEME_DEBUT#[Analyse des subventions]**
**#ATTENDU_DEBUT#Objet de la requête proposée :**
Identifier les bénéficiaires les plus importants pour chaque tranche de subventions, ainsi que les informations liées à ces bénéficiaires.
**#ATTENDU_FIN#**
**#COMP_FIN#**

/* 
SELECT DISTINCT
    '0-10000' AS tranche,
    IdAttrib

In [18]:
print(llm_response['choices'][0]['message']['content'])

/* [Analyse temporelle des organisations] : Evolution du nombre d'organisations enregistrées par mois */
SELECT
    strftime('%Y-%m', "created_at") AS date_created,
    COUNT(DISTINCT "organization") AS nombre_organisations
FROM example_dataset
GROUP BY 1
ORDER BY 1;





In [39]:
llm = Llama.from_pretrained(
	repo_id="bartowski/gemma-2-9b-it-GGUF",
	filename="gemma-2-9b-it-Q5_K_S.gguf",
 n_ctx=150000,
 verbose=False,
 n_gpu_layers=-1
)


In [ ]:
llm_response = llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": prompt_analyse
		}
	]
)
print(llm_response['choices'][0]['message']['content'])

Exception ignored on calling ctypes callback function: <function llama_log_callback at 0x130eb25f0>
Traceback (most recent call last):
  File "/Users/ali/env/projects/localllama/.venv/lib/python3.10/site-packages/llama_cpp/_logger.py", line 29, in llama_log_callback
    @llama_cpp.llama_log_callback
KeyboardInterrupt: 
